In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Divide de texto em chunks
from langchain_community.embeddings import OllamaEmbeddings # Gera embeddings transforma texto em vetores numéricos
from langchain_community.llms import Ollama # Modelo de linguagem para geração de respostas para Ollama
from langchain_community.vectorstores import Chroma # Banco de dados vetorial para armazenar embeddings
from langchain_community.document_loaders import PyPDFLoader # Carregador de documentos PDF
from langchain.chains.question_answering import load_qa_chain # Cria uma cadeia de perguntas e respostas faz a combinação de embeddings e modelo de linguagem

In [36]:
import os # Importa o módulo os para interagir com o sistema operacional para ser o arquivo e pegar a API key

In [37]:
# Carregar os modelos(Embeddings e LLM)
embeddings_model = OllamaEmbeddings(model='nomic-embed-text') # Modelo de embeddings do Ollama
llm_model = Ollama(model='gemma3:4b') # Modelo de LLM do Ollama

In [38]:
# Carregar o PDF
pdf_loader = PyPDFLoader("yan.pdf", extract_images=False) # Carregador de documentos PDF
pages = pdf_loader.load_and_split() # Divide o PDF em páginas


In [ ]:
# Separar em Chucks (Dividir o texto em pedaços menores)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # Tamanho máximo de cada chunk
    chunk_overlap = 20, # Overlap entre os chunks para manter o contexto
    length_function = len # Função para calcular o comprimento do texto
)

chunks = text_splitter.split_documents(pages) # Divide as páginas em chunks

In [40]:
# Salvar no Vector DB (Chroma)
vector_db = Chroma.from_documents(chunks, embeddings_model, persist_directory="chroma_db") # Cria um banco de dados vetorial a partir dos chunks
vector_db.persist() # Persiste o banco de dados vetorial

/var/folders/5l/x5tjl9qn4kq7zrm6vh3twr300000gn/T/ipykernel_51390/167552940.py:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist() # Persiste o banco de dados vetorial


In [41]:
#Carregar o Vector DB (Chroma)
vector_db = Chroma(persist_directory="chroma_db", embedding_function=embeddings_model) # Carrega o banco de dados vetorial persistido

# Load the retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3}) # Retorna os 3 chunks mais similares

#construção da cadeia de prompt para chamada do LLM
chain = load_qa_chain(llm_model, chain_type="stuff") # Cria uma cadeia de prompt para chamada do LLM


In [42]:
def pergunta(question):
    context = retriever.get_relevant_documents(question) # Busca os chunks mais similares à pergunta
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True)) ['output_text'] # Gera a resposta com base nos chunks mais similares
    return answer

In [ ]:
user_question = input("Digite sua pergunta: ")
answer = pergunta(user_question)
print("Resposta:",answer)